# Train the LSTM model for CANClassify

In [ ]:
from main import *

In [ ]:
labels = [
    'velocity_x', # velocity in the forward direction
    'acceleration_x', # acceleration in the forward direction
    'acceleration_y', # acceleration in the horizontal direction
    'steer_angle',
    'steer_angle_rate',
    'brake_pedal', # continuous value for how much it is pressed
    'gas_pedal', # continuous value for how much it is pressed
    'radar_long', # longitudinal
    'radar_lat', # latitudinal
    'radar_rel_vel', # relative velocity in the forward direction
    'radar_rel_acc', # relative acceleration in the forward direction
    'checksum',
]

In [4]:
with open("data/data_x", "rb") as f:
    X = pickle.load(f)

with open("data/data_y", "rb") as f:
    Y = pickle.load(f)

## Make the network

- Input layer of size 716 * 100
- Layer of size 500
- LSTM layer 200 (give sequence to next layer)
- LSTM layer 200 (dropout 0.1 to ->)
- Layer of size 100 (dropout 0.1 to ->)
- Layer of size 50
- Output layer (raw probabilities, no softmax)

In [35]:
import numpy as np
np.random.seed(285)

In [36]:
temp = list(zip(X, Y))
np.random.shuffle(temp)

X, Y = zip(*temp)

del temp

In [37]:
split_index = int(len(X)*0.80)

trainX = np.array(X[:split_index])
testX = np.array(X[split_index:])

trainy = np.array(Y[:split_index])
testy = np.array(Y[split_index:])

In [38]:
assert Y[0].shape[0] == len(labels)

## Only run training this once

In [39]:
n_timesteps, n_features = X[0].shape
n_outputs = Y[0].shape[0]

model = keras.Sequential()
model.add(layers.Dense(500, input_shape=(n_timesteps,n_features)))
model.add(layers.LSTM(256, input_shape=(500,), return_sequences=True))
model.add(layers.LSTM(256, input_shape=(500,)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(100))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(50))


# if using non softmax:
# model.add(layers.Dense(n_outputs))
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['MeanSquaredError'])

# if using softmax
# model.add(layers.Dense(n_outputs, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# if using multiclass multilabel
model.add(layers.Dense(n_outputs, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100, 500)          358500    
                                                                 
 lstm (LSTM)                 (None, 100, 256)          775168    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               25700     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 50)                5

In [40]:
history = model.fit(trainX, trainy, epochs=6, batch_size=32, verbose=1)

Epoch 1/6
907/907 [==============================] - 2246s 2s/step - loss: 0.2093 - accuracy: 0.4142
Epoch 2/6
907/907 [==============================] - 2267s 2s/step - loss: 0.1494 - accuracy: 0.6172
Epoch 3/6
907/907 [==============================] - 2243s 2s/step - loss: 0.1227 - accuracy: 0.7013
Epoch 4/6
907/907 [==============================] - 2174s 2s/step - loss: 0.0994 - accuracy: 0.7623
Epoch 5/6
907/907 [==============================] - 2118s 2s/step - loss: 0.0881 - accuracy: 0.7928
Epoch 6/6
907/907 [==============================] - 2102s 2s/step - loss: 0.0810 - accuracy: 0.8118


In [42]:
model.save(f'output/convolve_lstm_multilabel_model.tfsmf')

INFO:tensorflow:Assets written to: output/convolve_lstm_multilabel_model.tfsmf\assets


INFO:tensorflow:Assets written to: output/convolve_lstm_multilabel_model.tfsmf\assets
